## <center> **GLM for Loss Reserving** #

This script supports only one of the following GLM disrtibutions:
    
- Poisson
- Gamma
- Inverse Gaussian
    
---

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

from glm import *
pd.options.display.float_format = '{:.4f}'.format

In [2]:
# select model distribution

while True:
    glm = input("Enter model distribution: ").title()
    if glm == "Poisson":
        distribution = sm.families.Poisson(link=sm.families.links.log()) # Log()
        break
    elif glm == "Gamma":
        distribution = sm.families.Gamma(link=sm.families.links.log()) # Log()
        break
    elif glm == "Inverse Gaussian":
        distribution = sm.families.InverseGaussian(link=sm.families.links.inverse_squared()) # InversedSquared()
        break
    else:
        print("""Ups! That is not a valid distribution :(
        Try again...""")
        pass

Enter model distribution:  gamma


In [3]:
# import data
sheet_name = input("sheet name:")

X_table = import_triangle("triangles.xlsx", sheet_name, "table") # incremental observed triangle
X_triangle = import_triangle("triangles.xlsx", sheet_name, "triangle") # cumulative observed triangle

print(X_triangle)

sheet name: CAS


          1            2            3           4          5
1   7017487 4151522.0000  999910.0000 215639.0000 52760.0000
2   8954484 5091234.0000 1037206.0000 336748.0000        nan
3   9155776 6215702.0000 1086998.0000         nan        nan
4  10394069 6190043.0000          nan         nan        nan
5  12755243          nan          nan         nan        nan


In [4]:
X = pd.get_dummies(X_table, columns=['dev_year', 'acc_year'], drop_first=True).iloc[:,1:] # X (design matrix w/out constant)
y = X_table['loss'] # y (target vector)

# GLM #
formula = gen_formula(X)
model = sm.GLM.from_formula(formula,
                            data=pd.concat([y, X], axis=1),
                            family=distribution)
                                                           
model = model.fit()
print(model.summary(), formula, sep="\n")

                 Generalized Linear Model Regression Results                  
Dep. Variable:                   loss   No. Observations:                   15
Model:                            GLM   Df Residuals:                        6
Model Family:                   Gamma   Df Model:                            8
Link Function:                    log   Scale:                        0.010660
Method:                          IRLS   Log-Likelihood:                -202.41
Date:                Mon, 21 Aug 2023   Deviance:                     0.062640
Time:                        19:59:47   Pearson chi2:                   0.0640
No. Iterations:                    12                                         
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     15.7651      0.071    222.196      0.0

In [5]:
# generates incremental loss data table (with GLM eq.)
adj_table = gen_adj_table(model) 

m_triangle = to_triangular_form(adj_table) # incremental adj triangle
M_triangle = to_cum_triangle(m_triangle) # cumulative adj triangle
print(m_triangle)

              1            2            3           4          5
1  7025480.0299 4272417.0210  879915.2000 241459.8777 52760.0000
2  8851427.9541 5382833.8120 1108608.3749 304216.1819 66472.5167
3  9351880.4049 5687174.8051 1171288.1799 321416.3144 70230.8181
4 10286428.0760 6255502.8567 1288336.7940 353535.9369 77249.0909
5 12755243.0000 7756867.4407 1597546.6656 438387.0428 95789.4148


In [6]:
# loss reserve by AY
ultimate, ibnr = calculate_loss(M_triangle)

print(f"IBNR: {ibnr.sum()}",
      f"Ultimate: {ultimate.sum()}",
      sep="\n")

IBNR: 12065832.034934722
Ultimate: 85692467.80890052


---